In [1]:
import time
from datetime import date
from dataclasses import dataclass
import random

import numpy as np
import pandas as pd

import psycopg2 as pg2

from faker import Faker
from passlib.context import CryptContext

# to ignore warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('passlib').setLevel(logging.ERROR)

from db_config import get_db_config

## Classes

In [40]:
@dataclass
class Book:
    isbn: str
    title: str
    author: str
    publication_year: int
    publisher: str
    image_url: str

In [3]:
@dataclass
class User:
    # user_id: int
    username: str
    email: str
    password: str

In [4]:
@dataclass
class Rating:
    isbn: str
    user_id: int
    rating: int

In [5]:
pwd_ctx = CryptContext(schemes='bcrypt', deprecated='auto')

class Hash():
    def bcrypt(password: str):
        return pwd_ctx.hash(password)

## Functions

In [6]:
def insert_data(conn, query, data, table_name):
    try:
        with conn.cursor() as cur:
            if isinstance(data, list):
                cur.executemany(query, data)
            else:
                cur.execute(query, data)
                
            conn.commit()
            print(f"Data successfully inserted into table '{table_name}'!")
            
    except (Exception, pg2.DatabaseError) as error:
        print(f"Error inserting data into table '{table_name}':", error)
        conn.rollback()

In [7]:
def insert_autoincrement_data(conn, query, data, table_name):
    try:
        with conn.cursor() as cur:
            cur.execute(query, data)
            res = cur.fetchone()[0]  # Fetch the returned user_id
            conn.commit()
            # print(f"Data successfully inserted into table '{table_name}'!")
            return res
            
    except (Exception, pg2.DatabaseError) as error:
        print(f"Error inserting data into table '{table_name}':", error)
        conn.rollback()

### user

In [8]:
def generate_random_user(faker, user_id):
    """
    Generates a random user using Faker library.

    Parameters:
    - faker: Faker instance
    - user_id: int, ID of the user

    Returns:
    - User: instance of User class with random data
    """
    # Generate random data using Faker
    username = f"{faker.first_name()}{user_id}"
    # username = f"{faker.user_name()}{user_id}"
    email = f"{username}@gmail.com"
    password = Hash.bcrypt("12345")

    # Create and return User instance
    return User(username=username, email=email, password=password)  

In [9]:
def insert_into_user_table(conn, user):
    # Construct the SQL query
    query = """
        INSERT INTO "user" (
            username, email, password
        )
        VALUES (%s, %s, %s)
        RETURNING user_id;
    """
    data = (
            user.username,
            user.email,
            user.password
    )
    return insert_autoincrement_data(conn, query, data, 'user')   

In [10]:
def delete_from_user_table(conn, user_id):
    try:
        with conn.cursor() as cur:
            query = """
                DELETE FROM "user" 
                WHERE user_id = %s;
            """
            cur.execute(query, (user_id,))
            conn.commit()
    except (Exception, pg2.DatabaseError) as error:
        print(f"Error deleting data from table 'user': {error}")
        conn.rollback()

### book

In [43]:
def create_book(row):
    return Book(
        isbn = row['isbn'],
        title = row['title'],
        author = row['author'],
        publication_year = row['publication_year'],
        publisher = row['publisher'],
        image_url = row['image_url']
    )

In [36]:
def insert_into_book_table(conn, books):
    # Construct the SQL query
    query = """
        INSERT INTO book (
            isbn, title, author, publication_year, publisher, image_url
        )
        VALUES (%s, %s, %s, %s, %s, %s);
    """
    data = [
        (
            book.isbn,
            book.title,
            book.author,
            book.publication_year, 
            book.publisher, 
            book.image_url
        )
        for book in books
    ]
    if len(data) == 1:
        data = data[0]
    insert_data(conn, query, data, 'book')

## Read and check datasets

In [11]:
books_df = pd.read_csv('../data/db_data/Books.csv')
users_df = pd.read_csv('../data/db_data/Users.csv')
ratings_df = pd.read_csv('../data/db_data/Ratings.csv')

print(f"books_df.shape = {books_df.shape}")
print(f"users_df.shape = {users_df.shape}")
print(f"ratings_df.shape = {ratings_df.shape}")

books_df.shape = (271358, 6)
users_df.shape = (278858, 3)
ratings_df.shape = (1031136, 3)


In [12]:
books_df['isbn_len'] = books_df['isbn'].apply(lambda x: len(x))
ratings_df['isbn_len'] = ratings_df['isbn'].apply(lambda x: len(x))

# There should be only 1 value - 10
print(f"books_df['isbn_len'].unique() = {books_df['isbn_len'].unique()}")
print(f"ratings_df['isbn_len'].unique() = {ratings_df['isbn_len'].unique()}\n")

books_df.drop('isbn_len', axis=1, inplace=True)
ratings_df.drop('isbn_len', axis=1, inplace=True)

# len(ratings_df) should be equal to 1031136
print(f"len(ratings_df) = {len(ratings_df)}")

books_df['isbn_len'].unique() = [10]
ratings_df['isbn_len'].unique() = [10]

len(ratings_df) = 1031136


## Datasets Analysis for SQL tables

### books_df

In [13]:
books_df.head(2)

,isbn,title,author,publication_year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [14]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271358 entries, 0 to 271357
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   isbn              271358 non-null  object
 1   title             271358 non-null  object
 2   author            271358 non-null  object
 3   publication_year  271358 non-null  int64 
 4   publisher         271356 non-null  object
 5   image_url         271358 non-null  object
dtypes: int64(1), object(5)
memory usage: 12.4+ MB


In [15]:
print(f"books_df['publication_year'].min() = {books_df['publication_year'].min()}; books_df['publication_year'].max() = {books_df['publication_year'].max()}")

books_df['publication_year'].min() = 0; books_df['publication_year'].max() = 2023


In [17]:
books_df_filled = books_df.fillna('')

# Determine the maximum length of the record in each column with object data type
max_length_series = books_df_filled.select_dtypes(include=['object']).applymap(len).max()

max_length_series

isbn          10
title        256
author       143
publisher    134
image_url     60
dtype: int64

In [18]:
books_df[books_df['title'].apply(lambda title: len(title) >= 100)]

,isbn,title,author,publication_year,publisher,image_url
22,1879384493,If I'd Known Then What I Know Now: Why Not Lea...,J. R. Parrish,2003,Cypress House,http://images.amazon.com/images/P/1879384493.0...
101,0446677450,"Rich Dad, Poor Dad: What the Rich Teach Their ...",Robert T. Kiyosaki,2000,Warner Books,http://images.amazon.com/images/P/0446677450.0...
106,0684822733,"Love, Miracles, and Animal Healing : A heartwa...",Pam Proctor,1996,Fireside,http://images.amazon.com/images/P/0684822733.0...
159,0061030147,Cybill Disobedience: How I Survived Beauty Pag...,Cybill Shepherd,2001,Avon Books,http://images.amazon.com/images/P/0061030147.0...
276,0380730448,The Adrian Mole Diaries : The Secret Diary of ...,Sue Townsend,1997,Perennial,http://images.amazon.com/images/P/0380730448.0...
...,...,...,...,...,...,...
271241,0321196775,A Brief History of Western Civilization : The ...,Mark Kishlansky,2004,Longman,http://images.amazon.com/images/P/0321196775.0...
271283,1584857447,True Stories: Girls' Inspiring Stories of Cour...,Trula Magruder,2003,American Girl,http://images.amazon.com/images/P/1584857447.0...
271319,3453047192,Die amerikanische Zumutung: PlÃ¤doyers gegen d...,Rolf Winter,1990,W. Heyne,http://images.amazon.com/images/P/3453047192.0...
271327,8420614556,Lewis Carroll: A Traves Del Espejo Y Lo Que Al...,Lewis Carroll,1986,Lectorum Pubns (Adult),http://images.amazon.com/images/P/8420614556.0...


In [19]:
example_titles_no = 10
example_long_titles = list(books_df[books_df['title'].apply(lambda title: len(title) >= 200)][:example_titles_no]['title'])

[print(f"{index + 1}) title length: {len(example_long_titles[index])}; title: {example_long_titles[index]}") for index in range(example_titles_no)]

1) title length: 200; title: Frankenstein: Complete, Authoritative Text With Biographical, Historical, and Cultural Contexts, Critical History, and Essays from Contemporary Critic ... tive (Case Studies in Contemporary Criticism)
2) title length: 200; title: The Girlfriends' Guide to Surviving the First Year of Motherhood: Wise and Witty Advice on Everything from Coping With Postpartum Mood Swings to Salva ...  to  Fitting into That Favorite Pair of Jeans
3) title length: 200; title: The House of Mirth: Complete, Authoritative Text With Biographical and Historical Contexts, Critical History, and Essays from Five Contemporary Critic ... ives (Case Studies in Contemporary Criticism)
4) title length: 201; title: The American Country Inn And Bed &amp; Breakfast Cookbook, Volume I : More than 1,700 crowd-pleasing recipes from 500 American Inns (American Country Inn &amp; Bed &amp; Breakfast Cookbook (Hardcover))
5) title length: 204; title: The New Strong's exhaustive concordance of the Bib

[None, None, None, None, None, None, None, None, None, None]

In [20]:
example_authors_no = 5

example_long_authors = list(books_df[books_df['author'].apply(lambda author: len(author) >= 100)][:example_authors_no]['author'])
example_authors_no = len(example_long_authors)

[print(f"{index + 1}) author name length: {len(example_long_authors[index])}; author name: {example_long_authors[index]}") for index in range(example_authors_no)]

1) author name length: 122; author name: Based on the Larger Work'the Book of Garden Flowers'by G.a.R.Phillips Compiled by David Pycraft Illustrated by Joan Lupton
2) author name length: 107; author name: Staceyann Chin, Dot Antoniades, Aileen Reyes, Meaghan Williams, Miriam Stanley, Amy Ouzoonian Kyrce Swenson
3) author name length: 143; author name: Dale D. Johnson, Theodore Clymer, Roselmina Indrisano, Richard L. Venezky, James F. Baumann, Elfrieda Hiebert, and Marian Toth P. David Pearson


[None, None, None]

In [21]:
example_publishers_no = 5

example_long_publishers = list(books_df[books_df['publisher'].apply(lambda publisher: len(publisher) >= 100 if type(publisher) != float else False)][:example_publishers_no]['publisher'])
example_publishers_no = len(example_long_publishers)

[print(f"{index + 1}) publisher name length: {len(example_long_publishers[index])}; publisher name: {example_long_publishers[index]}") for index in range(example_publishers_no)]

1) publisher name length: 121; publisher name: Instituto Nacional de InvestigaÃ§Ã£o CientÃ­fica, Centro de Estudos ClÃ¡ssicos e HumanÃ­sticos da Universidade de Coimbra
2) publisher name length: 134; publisher name: Published by Natural Heritage/Natural History Inc. for the Ontario Heritage Foundation, Ontario Ministry of Culture and Communications
3) publisher name length: 115; publisher name: Narcissus Publications in association with Central Europe Review and Central and East European New Media Initiative


[None, None, None]

### users_df

In [22]:
user_ids = users_df['user_id'].values
print(f"type(user_ids) = {type(user_ids)}; len(user_ids) = {len(user_ids)}")

type(user_ids) = <class 'numpy.ndarray'>; len(user_ids) = 278858


In [23]:
check_user_ids_array = np.arange(1, len(users_df)+ 1)
print(f"type(check_user_ids_array) = {type(check_user_ids_array)}; len(check_user_ids_array) = {len(check_user_ids_array)}")

type(check_user_ids_array) = <class 'numpy.ndarray'>; len(check_user_ids_array) = 278858


In [24]:
np.array_equal(user_ids, check_user_ids_array)

True

## Connect to the PostgreSQL database

In [25]:
config_dict = get_db_config()
config_dict.keys()

dict_keys(['database', 'user', 'password'])

In [54]:
conn = pg2.connect(database=config_dict['database'], user=config_dict['user'], password=config_dict['password'])

# View some data from db
with conn.cursor() as cur:
    cur.execute('SELECT * FROM book')
    data = cur.fetchone()
    print(f"data = {data}")

data = None


## Filling the database tables

In [49]:
faker = Faker()
BATCH_SIZE = 200

### user table

#### Test insertion

In [28]:
for index, row in users_df.iterrows():
    user = generate_random_user(faker, row['user_id'])
    print(f"random_user = {user}")
    user_id = insert_into_user_table(conn, user)
    print(f"user_id = {user_id}")
    if user_id != row['user_id']:
        print(f"Error inserting data into table 'user': user_id ({user_id}) != row['user_id'] ({row['user_id']}).")
        break
    break

random_user = User(username='Oscar1', email='Oscar1@gmail.com', password='$2b$12$E0RjuejjJiqawWkw6SnrOOzfxRknyKwSBC1AcKqTjXrdlhPUC9o3u')
user_id = 1


In [29]:
start_index = 1
end_index = 5
current_users_df = users_df[start_index:end_index]
for index, row in current_users_df.iterrows():
    user = generate_random_user(faker, row['user_id'])
    print(f"random_user = {user}")
    user_id = insert_into_user_table(conn, user)
    print(f"user_id = {user_id}")
    if user_id != row['user_id']:
        print(f"Error inserting data into table 'user': user_id ({user_id}) != row['user_id'] ({row['user_id']}).")
        break
    print("")

random_user = User(username='Elizabeth2', email='Elizabeth2@gmail.com', password='$2b$12$37D/sQU.fu37Qldbs1ThZuFGwDz8YlYXQvByVrGIbTsGmy667EvSS')
user_id = 2

random_user = User(username='Jonathan3', email='Jonathan3@gmail.com', password='$2b$12$R78x9OFUQLL361jZ7lcDduMgkqejUY1tNaOTi18/jNdx8byq2CA1e')
user_id = 3

random_user = User(username='Christopher4', email='Christopher4@gmail.com', password='$2b$12$AkgAMzEXsomEsZET3/no5ucueKQW8oCUPnL1PhxOSesFKpqp/bw4G')
user_id = 4

random_user = User(username='Samuel5', email='Samuel5@gmail.com', password='$2b$12$VFWsILN4TF4hNpVwcHk7uOgTW0wbvo.VzYlNe3KSj5rnP0bnChju.')
user_id = 5



In [30]:
start_index = 5
end_index = 10
current_users_df = users_df[start_index:end_index]
current_users_df

,user_id,location,age
5,6,"santa monica, california, usa",61.0
6,7,"washington, dc, usa",NaN
7,8,"timmins, ontario, canada",NaN
8,9,"germantown, tennessee, usa",NaN
9,10,"albacete, wisconsin, spain",26.0


In [31]:
# View some data from db
with conn.cursor() as cur:
    cur.execute('SELECT * FROM "user"')
    data = cur.fetchall()
    print(f"data = {data}")

data = [(1, 'Oscar1', 'Oscar1@gmail.com', '$2b$12$E0RjuejjJiqawWkw6SnrOOzfxRknyKwSBC1AcKqTjXrdlhPUC9o3u'), (2, 'Elizabeth2', 'Elizabeth2@gmail.com', '$2b$12$37D/sQU.fu37Qldbs1ThZuFGwDz8YlYXQvByVrGIbTsGmy667EvSS'), (3, 'Jonathan3', 'Jonathan3@gmail.com', '$2b$12$R78x9OFUQLL361jZ7lcDduMgkqejUY1tNaOTi18/jNdx8byq2CA1e'), (4, 'Christopher4', 'Christopher4@gmail.com', '$2b$12$AkgAMzEXsomEsZET3/no5ucueKQW8oCUPnL1PhxOSesFKpqp/bw4G'), (5, 'Samuel5', 'Samuel5@gmail.com', '$2b$12$VFWsILN4TF4hNpVwcHk7uOgTW0wbvo.VzYlNe3KSj5rnP0bnChju.')]


#### Continue data insertion

In [50]:
start_index = 5
end_index = 1000
current_users_df = users_df[start_index:end_index]
data_insertion_start_time = time.time()
for index, row in current_users_df.iterrows():
    user = generate_random_user(faker, row['user_id'])
    user_id = insert_into_user_table(conn, user)
    if user_id != row['user_id']:
        print(f"Error inserting data into table 'user': user_id ({user_id}) != row['user_id'] ({row['user_id']}).")
        break

    if user_id % BATCH_SIZE == 0:
        print(f"{user_id}) {BATCH_SIZE} records successfully inserted into table 'user'!")
    
data_insertion_duration = time.time() - data_insertion_start_time
print(f"Data insertion time: {data_insertion_duration // 60: .0f}m {data_insertion_duration % 60: .0f}s\n")

200) 200 records successfully inserted into table 'user'!
400) 200 records successfully inserted into table 'user'!
600) 200 records successfully inserted into table 'user'!
800) 200 records successfully inserted into table 'user'!
1000) 200 records successfully inserted into table 'user'!
Data insertion time:  3m  44s



In [51]:
start_index = 1000
end_index = 2000
current_users_df = users_df[start_index:end_index]
data_insertion_start_time = time.time()
for index, row in current_users_df.iterrows():
    user = generate_random_user(faker, row['user_id'])
    user_id = insert_into_user_table(conn, user)
    if user_id != row['user_id']:
        print(f"Error inserting data into table 'user': user_id ({user_id}) != row['user_id'] ({row['user_id']}).")
        break

    if user_id % BATCH_SIZE == 0:
        print(f"{user_id}) {BATCH_SIZE} records successfully inserted into table 'user'!")
    
data_insertion_duration = time.time() - data_insertion_start_time
print(f"Data insertion time: {data_insertion_duration // 60: .0f}m {data_insertion_duration % 60: .0f}s\n")

1200) 200 records successfully inserted into table 'user'!
1400) 200 records successfully inserted into table 'user'!
1600) 200 records successfully inserted into table 'user'!
1800) 200 records successfully inserted into table 'user'!
2000) 200 records successfully inserted into table 'user'!
Data insertion time:  3m  45s



### book table

In [32]:
books_df.shape

(271358, 6)

In [41]:
books_df.head(2)

,isbn,title,author,publication_year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


#### Test insertion

In [44]:
for index, row in books_df.iterrows():
    book = create_book(row)
    print(f"book = {book}")
    insert_into_book_table(conn, [book])
    break

book = Book(isbn='0195153448', title='Classical Mythology', author='Mark P. O. Morford', publication_year=2002, publisher='Oxford University Press', image_url='http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg')
Data successfully inserted into table 'book'!


In [45]:
start_index = 1
end_index = 5
current_books_df = books_df[start_index:end_index]
for index, row in current_books_df.iterrows():
    book = create_book(row)
    print(f"book = {book}")
    insert_into_book_table(conn, [book])
    print("")

book = Book(isbn='0002005018', title='Clara Callan', author='Richard Bruce Wright', publication_year=2001, publisher='HarperFlamingo Canada', image_url='http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg')
Data successfully inserted into table 'book'!

book = Book(isbn='0060973129', title='Decision in Normandy', author="Carlo D'Este", publication_year=1991, publisher='HarperPerennial', image_url='http://images.amazon.com/images/P/0060973129.01.MZZZZZZZ.jpg')
Data successfully inserted into table 'book'!

book = Book(isbn='0374157065', title='Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It', author='Gina Bari Kolata', publication_year=1999, publisher='Farrar Straus Giroux', image_url='http://images.amazon.com/images/P/0374157065.01.MZZZZZZZ.jpg')
Data successfully inserted into table 'book'!

book = Book(isbn='0393045218', title='The Mummies of Urumchi', author='E. J. W. Barber', publication_year=1999, publisher='W. W. Norton 

#### Continue data insertion

In [52]:
start_index = 5
total_records = 1000 # len(books_df)

iteration = 0
data_insertion_start_time = time.time()

while start_index < total_records:
    end_index = min(start_index + BATCH_SIZE, total_records)
    batch_df = books_df.iloc[start_index:end_index]
    books_list = []
    
    iteration_start_time = time.time()
    for index, row in batch_df.iterrows():
         
        book_obj = create_book(row)
        books_list.append(book_obj)
    
    insert_into_book_table(conn, books_list)
    
    iteration_duration = time.time() - iteration_start_time
    print(f"\nIteration №{iteration + 1} duration: {iteration_duration // 60: .0f}m {iteration_duration % 60: .3f}s")
    
    iteration += 1
    start_index += BATCH_SIZE

data_insertion_duration = time.time() - data_insertion_start_time
print(f"Data insertion time: {data_insertion_duration // 60: .0f}m {data_insertion_duration % 60: .3f}s\n")

Data successfully inserted into table 'book'!

Iteration №1 duration:  0m  0.041s
Data successfully inserted into table 'book'!

Iteration №2 duration:  0m  0.039s
Data successfully inserted into table 'book'!

Iteration №3 duration:  0m  0.038s
Data successfully inserted into table 'book'!

Iteration №4 duration:  0m  0.038s
Data successfully inserted into table 'book'!

Iteration №5 duration:  0m  0.037s
Data insertion time:  0m  0s



In [53]:
conn.close()